In [79]:
!wget https://raw.githubusercontent.com/yuchen118125/data/master/KLAC.csv

--2020-04-23 17:39:52--  https://raw.githubusercontent.com/yuchen118125/data/master/KLAC.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 175696 (172K) [text/plain]
Saving to: ‘KLAC.csv.1’

KLAC.csv.1          100%[===================>] 171.58K  --.-KB/s    in 0.03s   

2020-04-23 17:39:52 (4.93 MB/s) - ‘KLAC.csv.1’ saved [175696/175696]



In [90]:
import pandas as pd
import numpy as np
import datetime
import tqdm #進度條
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline
data = pd.read_csv('KLAC.csv')
Date = data.get("Date")
Date = pd.to_datetime(Date)
Open = data.get("Open")
High = data.get("High")
Low = data.get("Low")
Close = data.get("Close")

dict = {"Date": Date,
        "open": Open,
        "high": High,
        "low": Low, 
        "close": Close,
        "return":0,
       }
df = pd.DataFrame(dict)
df.set_index('Date', inplace=True)
df = df.dropna()
print("總共:%d個交易時間點" % (len(Date)))
print("去除空白值後剩下:%d個交易時間點" % (len(df)))

總共:2515個交易時間點
去除空白值後剩下:2515個交易時間點


In [ ]:
def create_labels(df,col_name,window_size=11):
  row_counter = 0
  total_rows = len(df)
  labels = np.zeros(total_rows)
  labels[:]=np.nan
  while row_counter <total_rows:
    if row_counter>=window_size-1:
      window_begin=row_counter-(window_size-1)
      window_end=row_counter
      window_middle=(window_begin+window_end)/2
      min_=np.inf
      min_index=-1
      max_=-np.inf
      max_index=-1
      for i in range(window_begin,window_end+1):
        price = df.iloc[i][col_name]
        if price<min_:
          min_=price
          min_index=i
        elif price>max_:
          max_=price
          max_index=i
      if max_index == window_middle:
        labels[row_counter] = 2
      elif min_index == window_middle:
        labels[row_counter] = 0
      else:
        labels[row_counter] = 1
    row_counter = row_counter+1
  return labels
df['return'] = create_labels(df,'close')

In [92]:
result = pd.value_counts(df['return'])
print(result)
len(df)

1.0    2170
2.0     178
0.0     157
Name: return, dtype: int64


2515

In [93]:
from keras.utils import np_utils
#每張圖包含幾天的資料
time_period = 20
#預設幾日後的漲跌
day = 1

cnn_x = []
cnn_y = []
indexs = []

for i in tqdm.tqdm_notebook(range(0, len(df)-time_period-1-day)):
    r = df['return'].iloc[i+time_period-1+day]
    cnn_y.append(r)
    indexs.append(df.index[i+time_period-1])
    
cnn_y = np.array(cnn_y)
indexes = np.array(indexs)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
#2010~2018年為訓練和測試，2019年則為驗證
import datetime
#cnn_y_t = cnn_y[indexes < datetime.datetime(2019,1,1)]
cnn_y_validation = cnn_y[indexes > datetime.datetime(2019,1,1)]

In [95]:
df['close'][2267:]

Date
2019-01-07     92.599998
2019-01-08     91.129997
2019-01-09     93.480003
2019-01-10     93.709999
2019-01-11     94.029999
                 ...    
2019-12-23    177.110001
2019-12-24    178.110001
2019-12-26    178.750000
2019-12-27    178.520004
2019-12-30    178.369995
Name: close, Length: 248, dtype: float64

In [ ]:
f = df['close'][2267:]
a = []
b = []
for i in range(len(f)):
  a.append(f[i])
  b.append(cnn_y_validation[i])
a = np.array(a)
b = np.array(b)

S_dataset = pd.DataFrame({
    'price':a,    
    'signal':b,
}) 

In [ ]:
#起始資金10000美金，每次交易手續費9美金(以eToro為例)，stock是現在持有的股票數，previous上一部的狀態一樣則不動作。
total_money = 10000
Handling_fee = 9
stock = 0
previous = 1
for i in range(len(S_dataset)):
  total_money = total_money
  stock = stock
  previous = previous
  now = S_dataset['signal'][i]
  if now == previous:
    previous = now
  else:
    if now == 2:
      if stock!=0:
        total_money = total_money+(S_dataset['price'][i]*stock)
        total_money = total_money-Handling_fee
        stock=0
    elif now == 0:
      if total_money>S_dataset['price'][i]:
        stock = int(total_money/S_dataset['price'][i])
        total_money = total_money-(stock*S_dataset['price'][i])
    previous = now

In [ ]:
total = total_money+(S_dataset['price'][len(S_dataset)-1]*stock)-Handling_fee

In [99]:
#Return on investment (%)
print(((total-10000)/10000)*100)

94.87132369999995


In [ ]:
f = df['close'][12:]
a = []
b = []
for i in range(len(f)):
  a.append(f[i])
  b.append(cnn_y[i])
a = np.array(a)
b = np.array(b)

S_dataset = pd.DataFrame({
    'price':a,    
    'signal':b,
}) 

IndexError: ignored

In [ ]:
#起始資金10000美金，每次交易手續費9美金(以eToro為例)，stock是現在持有的股票數，previous上一部的狀態一樣則不動作。
total_money = 10000
Handling_fee = 9
stock = 0
previous = 1
for i in range(len(S_dataset)):
  total_money = total_money
  stock = stock
  previous = previous
  now = S_dataset['signal'][i]
  if now == previous:
    previous = now
  else:
    if now == 2:
      if stock!=0:
        total_money = total_money+(S_dataset['price'][i]*stock)
        total_money = total_money-Handling_fee
        stock=0
    elif now == 0:
      if total_money>S_dataset['price'][i]:
        stock = int(total_money/S_dataset['price'][i])
        total_money = total_money-(stock*S_dataset['price'][i])
    previous = now

In [ ]:
total = total_money+(S_dataset['price'][len(S_dataset)-1]*stock)-Handling_fee

In [ ]:
#Return on investment (%)
print(((total-10000)/10000)*100)

-53.12252359000008
